# Notebook Overview

1. **Setup**:
    - Import libraries and configure Polars settings.

2. **Data Loading**:
    - Load predictions, entries, stops, and patterns data from Parquet and JSON files.

3. **Data Mapping and Formatting**:
    - Define mappings for route IDs and status colors.
    - Format and correct time columns in predictions data.

4. **Data Processing**:
    - Join stops data to entries data to get stop names.
    - Filter and transform entries data to include only in-service records and calculate time differences.

5. **Subset and Join Operations**:
    - Filter data for specific route IDs and stops.
    - Perform join operations to calculate time differences for stops ahead.

6. **Descriptive Statistics**:
    - Generate and display descriptive statistics for time differences.


In [1]:
import polars as pl
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import json
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pl.enable_string_cache()
pl.Config().set_tbl_cols(100)
pl.Config().set_tbl_rows(25)

polars.config.Config

In [2]:
%config InteractiveShell.ast_node_interactivity = 'last_expr_or_assign'

In [3]:
#Map route id to correct route name
route_mapping = {
    3: "2L",
    4: "2R",
    33: "3",
    17: "10",
    18: "11",
    23: "12",
    12: "16",
    13: "17",
    14: "18",
    30: "19",
    29: "21",
    38: "21 Tripper",
    777: "777"
}

#Map hex color to prediction status
color_mapping = {
    "#cf1625": "Early",
    "#39B139": "On Time",
    "#D58803": "Little Late",
    "#f3e413": "Very Late"
}

{'#cf1625': 'Early',
 '#39B139': 'On Time',
 '#D58803': 'Little Late',
 '#f3e413': 'Very Late'}

In [4]:
preds = pl.read_parquet("./data/2024-09-preds.parquet")

preds = preds.with_columns(pl.col("statuscolor").alias("statusName"))
preds = preds.with_columns(pl.col("statusName").replace_strict(color_mapping))

#Format the schedule time to be a datetime object
preds = preds.with_columns(
    pl.col('receiveTime').dt.convert_time_zone('America/Chicago'),
    pl.col('schedule').alias('scheduleStr'),
    pl.col('time').alias('timeStr'),
    pl.col('schedule').str.to_time(format='%I:%M%p', strict=False),
    pl.col('time').str.to_time(format='%I:%M%p', strict=False)
)

preds = preds.with_columns(pl.col('receiveTime').dt.combine(pl.col('time')).alias('time'),
                    pl.col('receiveTime').dt.combine(pl.col('schedule')).alias('schedule'))

#Correct for time columns hat are off by 24 hours
preds = preds.with_columns(
    pl.when(pl.col('time') - pl.col('receiveTime') > pl.duration(hours=12))
    .then(pl.col('time') - pl.duration(hours=24))
    .when(pl.col('time') - pl.col('receiveTime') < pl.duration(hours=-12))
    .then(pl.col('time') + pl.duration(hours=24))
    .otherwise(pl.col('time'))
    .alias('time')
)

#Correct for schedule times columns that are off by 24 hours
preds = preds.with_columns(
    pl.when(pl.col('schedule') - pl.col('receiveTime') > pl.duration(hours=12))
    .then(pl.col('schedule') - pl.duration(hours=24))
    .when(pl.col('schedule') - pl.col('receiveTime') < pl.duration(hours=-12))
    .then(pl.col('schedule') + pl.duration(hours=24))
    .otherwise(pl.col('schedule'))
    .alias('schedule')
)

# preds = preds.with_columns(
#     (pl.col("time") - pl.col("receiveTime")).alias("predictedDiff")
# )

# preds = preds.with_columns(
#     (pl.col("schedule") - pl.col("receiveTime")).alias("scheduleDiff")
# )


stopID,patternStopID,timePoint,minutes,time,status,schedule,scheduleNumber,statuscolor,direction,equipmentID,routeID,receiveTime,captureTime,__index_level_0__,statusName,scheduleStr,timeStr
i64,i64,bool,i64,"datetime[μs, America/Chicago]",str,"datetime[μs, America/Chicago]",str,cat,cat,str,i64,"datetime[ns, America/Chicago]",datetime[ns],i64,str,str,str
806,10612,false,1,2024-08-30 19:00:00 CDT,"""07:00PM""",2024-08-31 08:01:00 CDT,"""8:00:00-17""","""#f3e413""","""Outbound""","""1102""",13,2024-08-31 06:36:01 CDT,2024-09-01 00:00:01,20,"""Very Late""","""08:01AM""","""07:00PM"""
807,10613,false,1,2024-08-30 19:00:00 CDT,"""07:00PM""",2024-08-31 08:01:00 CDT,"""8:00:00-17""","""#f3e413""","""Outbound""","""1102""",13,2024-08-31 06:36:01 CDT,2024-09-01 00:00:01,21,"""Very Late""","""08:01AM""","""07:00PM"""
808,10614,false,2,2024-08-30 19:01:00 CDT,"""07:01PM""",2024-08-31 08:02:00 CDT,"""8:00:00-17""","""#f3e413""","""Outbound""","""1102""",13,2024-08-31 06:36:01 CDT,2024-09-01 00:00:01,22,"""Very Late""","""08:02AM""","""07:01PM"""
536,10615,true,3,2024-08-30 19:02:00 CDT,"""07:02PM""",2024-08-31 08:03:00 CDT,"""8:00:00-17""","""#f3e413""","""Outbound""","""1102""",13,2024-08-31 06:36:01 CDT,2024-09-01 00:00:01,23,"""Very Late""","""08:03AM""","""07:02PM"""
545,10616,false,4,2024-08-30 19:03:00 CDT,"""07:03PM""",2024-08-31 08:04:00 CDT,"""8:00:00-17""","""#f3e413""","""Outbound""","""1102""",13,2024-08-31 06:36:01 CDT,2024-09-01 00:00:01,24,"""Very Late""","""08:04AM""","""07:03PM"""
805,10611,true,1,2024-08-31 19:00:00 CDT,"""07:00PM""",2024-08-31 18:00:00 CDT,"""18:00:00-17""","""#f3e413""","""Outbound""","""502""",13,2024-08-31 18:59:46 CDT,2024-09-01 00:00:01,0,"""Very Late""","""06:00PM""","""07:00PM"""
806,10612,false,2,2024-08-31 19:01:00 CDT,"""07:01PM""",2024-08-31 18:01:00 CDT,"""18:00:00-17""","""#f3e413""","""Outbound""","""502""",13,2024-08-31 18:59:46 CDT,2024-09-01 00:00:01,1,"""Very Late""","""06:01PM""","""07:01PM"""
807,10613,false,2,2024-08-31 19:01:00 CDT,"""07:01PM""",2024-08-31 18:01:00 CDT,"""18:00:00-17""","""#f3e413""","""Outbound""","""502""",13,2024-08-31 18:59:46 CDT,2024-09-01 00:00:01,2,"""Very Late""","""06:01PM""","""07:01PM"""
808,10614,false,3,2024-08-31 19:02:00 CDT,"""07:02PM""",2024-08-31 18:02:00 CDT,"""18:00:00-17""","""#f3e413""","""Outbound""","""502""",13,2024-08-31 18:59:46 CDT,2024-09-01 00:00:01,3,"""Very Late""","""06:02PM""","""07:02PM"""


In [5]:
#preds['scheduleDiff'].describe()

In [6]:
#preds['predictedDiff'].describe()

In [7]:
df = pl.read_parquet("./data/2024-09-entries.parquet")
df = df.with_columns(pl.col("routeID").replace_strict(route_mapping))


routeID,patternID,equipmentID,tripID,lat,lng,load,capacity,eLoad,blockID,nextStopID,nextStopETA,nextPatternStopID,h,lastStopID,lastPatternStopID,scheduleNumber,inService,onSchedule,trainID,receiveTime,aID,captureTime,direction,seq,lastStopExtID,nextStopExtID,nextStopPctProg,atStop,__index_level_0__
str,i64,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,datetime[ns],str,datetime[ns],cat,i64,i64,i64,f64,bool,i64
"""777""",33,"""1001""",null,41.93536,-88.77044,0,0,4,0,0,-1,0,197,491,11402,"""NIS""",false,null,0,2024-08-31 23:59:49,"""35467605078ef4d""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,13
"""777""",9998,"""1002""",null,41.93484,-88.72824,0,0,9,0,0,-1,0,78,0,0,"""NIS""",false,null,0,2024-08-31 15:14:57,"""3546760508796c6""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,14
"""777""",0,"""1003""",null,41.93277,-88.76676,0,0,0,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-30 04:47:18,"""3546760504518c0""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,15
"""777""",9998,"""1004""",null,41.93481,-88.7283,0,0,15,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-31 09:43:05,"""35467605056be89""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,16
"""777""",9998,"""1101""",null,41.93486,-88.7282,0,0,0,125,0,-1,0,351,0,0,"""NIS""",false,null,0,2024-08-31 05:17:48,"""3546760504edeed""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,17
"""17""",13,"""1102""",10058,41.93558,-88.76723,0,0,237,0,806,1139,10612,2,805,10611,"""8:00:00-17""",true,-658.0,11606,2024-08-31 11:36:01,"""35467605059a8e2""",2024-09-01 00:00:01,"""Outbound""",2,805,806,100.0,true,18
"""777""",9998,"""1103""",null,41.93481,-88.72816,0,0,0,0,0,-1,0,47,0,0,"""NIS""",false,null,0,2024-08-31 02:19:58,"""35467605044c796""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,19
"""777""",0,"""1401""",null,41.93539,-88.77016,0,0,0,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-01 19:06:17,"""3546760504ac22e""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,20
"""777""",0,"""1701""",null,41.9348,-88.72831,0,0,0,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-10 04:51:32,"""3546760504a404a""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,21


In [8]:
#Load stops data
file = open("./data/stops.json", "r")
stopsData = json.load(file)

stops = pl.DataFrame(stopsData['get_stops'])

rid,rsid,id,name,lat,lng,extID,shortName
i64,i64,i64,str,f64,f64,str,str
3,197057,449,"""Anderson Hall""",41.938465,-88.761086,"""449""","""449"""
3,197056,448,"""Barsema Hall""",41.939774,-88.761009,"""448""","""448"""
3,197054,446,"""Castle Dr. at College Ave.""",41.93412,-88.763115,"""446""","""446"""
3,197053,445,"""Castle Dr. at Montgomery""",41.9324,-88.762863,"""445""","""445"""
3,197044,436,"""Convocation Center Entrance 3""",41.936459,-88.785286,"""436""","""436"""
3,197428,820,"""DuSable Turnaround Rt 2L / 2R""",41.934803,-88.769974,"""""","""538"""
3,197055,447,"""Gilbert Way at Art Building""",41.935471,-88.762093,"""447""","""447"""
3,197074,466,"""Grant North""",41.938599,-88.777321,"""466""","""466"""
3,197058,450,"""Health Services on Lucinda Ave…",41.936481,-88.764244,"""450""","""450"""


In [9]:
#Load patterns data
pattern_mapping = {
    3: "2L",
    4: "2R",
    37: "3",
    17: "10",
    18: "11",
    23: "12",
    12: "16",
    13: "17",
    14: "18",
    33: "19",
    46: "21",
    45: "21 Tripper",
}

#Load patterns json
file = open("./data/patterns.json", "r")
patternsData = json.load(file)

patterns = pl.DataFrame(patternsData['get_patterns'])
patterns = patterns.with_columns(pl.col("id").replace_strict(pattern_mapping, default="None"))

id,name,extID,type,length,color,encLine,decLine,routes,routeNames,stations,stopIDs
str,str,str,i64,f64,str,str,list[null],list[i64],list[str],list[null],list[i64]
"""2L""","""Route 2L (Full Service)""","""2""",2,5.156097,"""#097138""","""eq}~Fxix|OM?a@@}A??fE?nEAbE?h@…",[],[3],"[""Route 2L""]",[],"[433, 465, … 820]"
"""2R""","""Route 2R Full Service""","""1""",1,5.083211,"""#FFD600""","""qk}~Fn{x|Ow@C{BhAoBp@Em@G}G@_I…",[],[4],"[""Route 2R""]",[],"[431, 451, … 432]"
"""3""","""Route 3 Full Service""","""3""",2,2.793036,"""#58F964""","""kq}~Fxix|Oa@??@sA?@?C?A}C???aC…",[],[33],"[""Route 3""]",[],"[431, 493, … 431]"
"""10""","""Route 10 Full Service""","""10""",3,5.564219,"""#782BC9""","""{q}~Fxix|OuBACfF@jH@xClFkBj@Sj…",[],[17],"[""Route 10""]",[],"[477, 432, … 431]"
"""11""","""Route 11 Full Service""","""11""",2,12.793737,"""#A07D5C""","""mq}~F~ix|O}A?QECKEaPAoACmQGGeR…",[],[18],"[""Route 11""]",[],"[492, 431, … 491]"
"""12""","""Route 12 Full Service""","""12""",2,39.210159,"""#3C8DBC""","""{~|~Fzex|O?dCrIA@qE@}Bx@u}@@mF…",[],[23],"[""Route 12""]",[],"[809, 453, … 575]"
"""16""","""Route 16 Full Service""","""16""",3,4.9,"""#A64598""","""y~|~Ftcx|OAfErI@@}H?W~@cbAFmAD…",[],[12],"[""Route 16""]",[],"[809, 536, … 811]"
"""17""","""Route 17 Full Service""","""17""",2,11.542563,"""#3E5BA6""","""ap}~Fjkx|OHOAYQGu@B_B?K}]?_M`B…",[],[13],"[""Route 17""]",[],"[805, 806, … 805]"
"""18""","""Route 18 Full Service""","""18""",4,14.597877,"""#FFAA00""","""}i`_G|no|OoA}Aa@]gAKCB[~BUp@c@…",[],[14],"[""Route 18""]",[],"[676, 835, … 685]"


In [10]:
stops.rename({"id": "nextStopID"}).select(["nextStopID", "name"]).unique().sort("nextStopID")

nextStopID,name
i64,str
431,"""Student Center"""
432,"""DuSable Turnaround Rt 5 / Rt 1…"
433,"""Lincoln Hall"""
436,"""Convocation Center Entrance 3"""
437,"""Lot C3 at Presidents Blvd"""
438,"""Norther View Circle"""
439,"""Lincoln Hwy. at High Meadows"""
440,"""Lincoln Hwy. at Culvers"""
441,"""Lincoln Hwy. at Precision Auto…"


In [11]:
#Left join stops to get stop names for nextStopID and lastStopID
df = df.join(stops.rename({"id": "nextStopID"}).select(["nextStopID", "name"]).unique(), on="nextStopID", how="left").rename({"name": "nextStopName"})
df = df.join(stops.rename({"id": "lastStopID"}).select(["lastStopID", "name"]).unique(), on="lastStopID", how="left").rename({"name": "lastStopName"})


routeID,patternID,equipmentID,tripID,lat,lng,load,capacity,eLoad,blockID,nextStopID,nextStopETA,nextPatternStopID,h,lastStopID,lastPatternStopID,scheduleNumber,inService,onSchedule,trainID,receiveTime,aID,captureTime,direction,seq,lastStopExtID,nextStopExtID,nextStopPctProg,atStop,__index_level_0__,nextStopName,lastStopName
str,i64,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,datetime[ns],str,datetime[ns],cat,i64,i64,i64,f64,bool,i64,str,str
"""777""",33,"""1001""",null,41.93536,-88.77044,0,0,4,0,0,-1,0,197,491,11402,"""NIS""",false,null,0,2024-08-31 23:59:49,"""35467605078ef4d""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,13,null,"""Normal Rd. at Northern Lane"""
"""777""",9998,"""1002""",null,41.93484,-88.72824,0,0,9,0,0,-1,0,78,0,0,"""NIS""",false,null,0,2024-08-31 15:14:57,"""3546760508796c6""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,14,null,null
"""777""",0,"""1003""",null,41.93277,-88.76676,0,0,0,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-30 04:47:18,"""3546760504518c0""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,15,null,null
"""777""",9998,"""1004""",null,41.93481,-88.7283,0,0,15,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-31 09:43:05,"""35467605056be89""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,16,null,null
"""777""",9998,"""1101""",null,41.93486,-88.7282,0,0,0,125,0,-1,0,351,0,0,"""NIS""",false,null,0,2024-08-31 05:17:48,"""3546760504edeed""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,17,null,null
"""17""",13,"""1102""",10058,41.93558,-88.76723,0,0,237,0,806,1139,10612,2,805,10611,"""8:00:00-17""",true,-658.0,11606,2024-08-31 11:36:01,"""35467605059a8e2""",2024-09-01 00:00:01,"""Outbound""",2,805,806,100.0,true,18,"""Lucinda Ave at Wirtz Dr Stop #…","""Holmes Student Center Stop #73…"
"""777""",9998,"""1103""",null,41.93481,-88.72816,0,0,0,0,0,-1,0,47,0,0,"""NIS""",false,null,0,2024-08-31 02:19:58,"""35467605044c796""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,19,null,null
"""777""",0,"""1401""",null,41.93539,-88.77016,0,0,0,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-01 19:06:17,"""3546760504ac22e""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,20,null,null
"""777""",0,"""1701""",null,41.9348,-88.72831,0,0,0,0,0,-1,0,0,0,0,"""NIS""",false,null,0,2024-08-10 04:51:32,"""3546760504a404a""",2024-09-01 00:00:01,null,-1,-1,-1,null,false,21,null,null


In [12]:
#Filter out the subset of data we want to work with
df = df.filter(
    (pl.col("inService"))
)

#Add stopChanged column
df = df.with_columns(
    (
        (pl.col("lastStopID") == pl.col("nextStopID").shift(1)).over(
            "equipmentID", order_by="receiveTime"
        )
    ).alias("stopChanged")
)

df = df.filter(pl.col("stopChanged")).with_columns(
    (pl.col("nextStopID").shift(1) == pl.col("lastStopID"))
    .over("equipmentID", order_by="receiveTime")
    .alias("nextToLast")
)

#Add timeDiff column
df = df.with_columns(
    (-pl.col("receiveTime").diff(-1).over("equipmentID", order_by="receiveTime")).alias(
        "timeDiff"
    )
).filter(pl.col("nextToLast"))


routeID,patternID,equipmentID,tripID,lat,lng,load,capacity,eLoad,blockID,nextStopID,nextStopETA,nextPatternStopID,h,lastStopID,lastPatternStopID,scheduleNumber,inService,onSchedule,trainID,receiveTime,aID,captureTime,direction,seq,lastStopExtID,nextStopExtID,nextStopPctProg,atStop,__index_level_0__,nextStopName,lastStopName,stopChanged,nextToLast,timeDiff
str,i64,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,datetime[ns],str,datetime[ns],cat,i64,i64,i64,f64,bool,i64,str,str,bool,bool,duration[ns]
"""19""",33,"""603""",10252,41.94231,-88.77307,0,0,8,0,616,1143,11390,360,709,11389,"""33-0831_1""",true,-4.0,11681,2024-09-01 00:03:09,"""35467605059b849""",2024-09-01 00:03:17,"""Outbound""",4,709,616,0.0,true,357,"""Annie Glidden Rd. at Varsity D…","""Annie Glidden Rd. & Loren Dr. …",true,true,20s
"""19""",33,"""603""",10252,41.94478,-88.77306,0,0,8,0,482,1144,11391,0,616,11390,"""33-0831_1""",true,-4.0,11681,2024-09-01 00:03:29,"""35467605059b849""",2024-09-01 00:03:32,"""Outbound""",5,616,482,26.68739,false,384,"""Blackhawk Rd. at Kimberly Dr.""","""Annie Glidden Rd. at Varsity D…",true,true,50s
"""19""",33,"""603""",10252,41.94629,-88.76997,0,0,8,0,483,1144,11392,2,482,11391,"""33-0831_1""",true,-4.0,11681,2024-09-01 00:04:19,"""35467605059b849""",2024-09-01 00:04:33,"""Outbound""",6,482,483,0.0,true,492,"""Blackhawk Rd. at Edgebrook Dr.""","""Blackhawk Rd. at Kimberly Dr.""",true,true,1m 10s
"""21""",46,"""72309""",null,41.99158,-88.68532,0,0,0,0,764,1145,11448,9,763,11447,"""19:00:00-21""",true,-2.0,11696,2024-09-01 00:04:33,"""354676050c83bea""",2024-09-01 00:04:48,"""Outbound""",4,763,764,0.0,true,537,"""Lucas St. at Opportunity House""","""Maple St. at Opportunity House…",true,true,1m 10s
"""2L""",3,"""Old_1501""",329,41.93639,-88.7736,0,0,33,0,465,1145,10482,271,433,10481,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:04:41,"""354676050595540""",2024-09-01 00:04:48,"""Outbound""",4,433,465,0.0,true,538,"""Stadium Dr. at New Hall""","""Lincoln Hall""",true,true,40s
"""16""",12,"""72307""",10046,41.93175,-88.75342,0,0,47,0,624,1146,10598,112,623,10597,"""12-0831_1""",true,-1.0,11594,2024-09-01 00:04:48,"""354676050c83bf0""",2024-09-01 00:05:03,"""Outbound""",4,623,624,0.0,true,563,"""Locust St. at 6th St.""","""Locust St. at 2nd St.""",true,true,2m 20s
"""2L""",3,"""Old_1501""",329,41.93746,-88.7754,0,0,33,0,466,1146,10483,0,465,10482,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:05:21,"""354676050595540""",2024-09-01 00:05:33,"""Outbound""",5,465,466,0.0,true,619,"""Grant North""","""Stadium Dr. at New Hall""",true,true,1m
"""21""",46,"""72309""",null,41.99374,-88.68572,0,0,0,0,765,1147,11449,100,764,11448,"""19:00:00-21""",true,-2.0,11696,2024-09-01 00:05:43,"""354676050c83bea""",2024-09-01 00:05:49,"""Outbound""",5,764,765,0.0,true,645,"""State St. at Sycamore Post Off…","""Lucas St. at Opportunity House""",true,true,1m 40s
"""10""",17,"""904""",10159,41.93984,-88.77303,0,0,101,0,529,1146,10691,360,528,10690,"""17-0831_1""",true,-4.0,11166,2024-09-01 00:05:45,"""35467605084242c""",2024-09-01 00:05:49,"""Outbound""",4,528,529,0.0,true,633,"""Varsity Blvd. at Pappas""","""Annie Glidden Rd at Crane Dr""",true,true,50s


In [13]:
subset = df.filter(
    pl.col("routeID") == "2L"
)

routeID,patternID,equipmentID,tripID,lat,lng,load,capacity,eLoad,blockID,nextStopID,nextStopETA,nextPatternStopID,h,lastStopID,lastPatternStopID,scheduleNumber,inService,onSchedule,trainID,receiveTime,aID,captureTime,direction,seq,lastStopExtID,nextStopExtID,nextStopPctProg,atStop,__index_level_0__,nextStopName,lastStopName,stopChanged,nextToLast,timeDiff
str,i64,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,datetime[ns],str,datetime[ns],cat,i64,i64,i64,f64,bool,i64,str,str,bool,bool,duration[ns]
"""2L""",3,"""Old_1501""",329,41.93639,-88.7736,0,0,33,0,465,1145,10482,271,433,10481,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:04:41,"""354676050595540""",2024-09-01 00:04:48,"""Outbound""",4,433,465,0.0,true,538,"""Stadium Dr. at New Hall""","""Lincoln Hall""",true,true,40s
"""2L""",3,"""Old_1501""",329,41.93746,-88.7754,0,0,33,0,466,1146,10483,0,465,10482,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:05:21,"""354676050595540""",2024-09-01 00:05:33,"""Outbound""",5,465,466,0.0,true,619,"""Grant North""","""Stadium Dr. at New Hall""",true,true,1m
"""2L""",3,"""Old_1501""",329,41.93859,-88.77699,0,0,33,0,467,1147,10484,270,466,10483,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:06:21,"""354676050595540""",2024-09-01 00:06:34,"""Outbound""",6,466,467,0.0,true,727,"""Stevenson North""","""Grant North""",true,true,1m 20s
"""2L""",3,"""Old_1501""",329,41.93785,-88.77995,0,0,33,0,468,1148,10485,270,467,10484,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:07:41,"""354676050595540""",2024-09-01 00:07:50,"""Outbound""",7,467,468,3.04295,true,862,"""Stevenson South""","""Stevenson North""",true,true,51s
"""2L""",3,"""Old_1501""",329,41.93645,-88.78108,0,0,33,0,436,1149,10486,182,468,10485,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:08:32,"""354676050595540""",2024-09-01 00:08:35,"""Outbound""",8,468,436,0.0,true,943,"""Convocation Center Entrance 3""","""Stevenson South""",true,true,40s
"""2L""",3,"""Old_1501""",329,41.93643,-88.78561,0,0,33,0,437,1150,10487,268,436,10486,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:09:12,"""354676050595540""",2024-09-01 00:09:20,"""Outbound""",9,436,437,14.69113,true,1024,"""Lot C3 at Presidents Blvd""","""Convocation Center Entrance 3""",true,true,30s
"""2L""",3,"""Old_1501""",329,41.93696,-88.78789,0,0,33,0,438,1151,10488,360,437,10487,"""19:00:00-2""",true,-1.0,11414,2024-09-01 00:09:42,"""354676050595540""",2024-09-01 00:09:51,"""Outbound""",10,437,438,23.72232,true,1078,"""Norther View Circle""","""Lot C3 at Presidents Blvd""",true,true,30s
"""2L""",3,"""Old_1501""",329,41.93976,-88.78796,0,0,33,0,439,1151,10489,295,438,10488,"""19:00:00-2""",true,0.0,11414,2024-09-01 00:10:12,"""354676050595540""",2024-09-01 00:10:21,"""Outbound""",11,438,439,1.242841,true,1132,"""Lincoln Hwy. at High Meadows""","""Norther View Circle""",true,true,2m 20s
"""2L""",3,"""Old_1501""",329,41.93068,-88.77951,0,0,33,0,440,1153,10490,90,439,10489,"""19:00:00-2""",true,-1.0,11414,2024-09-01 00:12:32,"""354676050595540""",2024-09-01 00:12:37,"""Outbound""",12,439,440,1.330905,true,1375,"""Lincoln Hwy. at Culvers""","""Lincoln Hwy. at High Meadows""",true,true,1m 10s


In [14]:
df1 = subset.filter(pl.col('nextStopID') == 465)
df2 = subset.filter(pl.col('nextStopID') == 468)

df2 = df2.with_columns(pl.col("receiveTime").alias("receiveTime_right"))
joined_df = df1.join_asof(df2, on="receiveTime", by='equipmentID', strategy='forward')

joined_df = joined_df.with_columns(
    (pl.col("receiveTime_right") - pl.col("receiveTime")).alias("timeDiff_3_stops_ahead")
)

joined_df['routeID', 'equipmentID', 'lastStopName', 'nextStopName_right', 'receiveTime', 'receiveTime_right', 'timeDiff_3_stops_ahead']

routeID,equipmentID,lastStopName,nextStopName_right,receiveTime,receiveTime_right,timeDiff_3_stops_ahead
str,str,str,str,datetime[ns],datetime[ns],duration[ns]
"""2L""","""Old_1501""","""Lincoln Hall""","""Stevenson South""",2024-09-01 00:04:41,2024-09-01 00:07:41,3m
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 00:34:35,2024-09-01 00:37:16,2m 41s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 01:03:34,2024-09-01 01:05:55,2m 21s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 01:33:46,2024-09-01 01:36:37,2m 51s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 02:04:05,2024-09-01 02:06:35,2m 30s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 02:33:37,2024-09-01 02:35:48,2m 11s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 03:02:47,2024-09-01 03:05:08,2m 21s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 03:34:16,2024-09-01 03:37:06,2m 50s
"""2L""","""1103""","""Lincoln Hall""","""Stevenson South""",2024-09-01 04:04:30,2024-09-01 04:07:20,2m 50s


In [15]:
mega_df = pl.read_parquet('./data/mega_df.parquet')

routeID,patternID,equipmentID,tripID,nextStopLat,nextStopLng,load,capacity,eLoad,blockID,nextStopID,nextStopETA,nextPatternStopID,h,lastStopID,lastPatternStopID,scheduleNumber,inService,onSchedule,trainID,receiveTime,aID,captureTime,direction,seq,lastStopExtID,nextStopExtID,nextStopPctProg,atStop,__index_level_0__,nextStopName,lat_right,lng_right,lastStopName,lastStopLat,lastStopLng,stopChanged,nextToLast,timeDiff,routeID_right,patternID_right,tripID_right,nextStopLat_right,nextStopLng_right,load_right,capacity_right,eLoad_right,blockID_right,nextStopID_right,nextStopETA_right,nextPatternStopID_right,h_right,lastStopID_right,lastPatternStopID_right,scheduleNumber_right,inService_right,onSchedule_right,trainID_right,aID_right,captureTime_right,direction_right,seq_right,lastStopExtID_right,nextStopExtID_right,nextStopPctProg_right,atStop_right,__index_level_0___right,nextStopName_right,lat_right_right,lng_right_right,lastStopName_right,lastStopLat_right,lastStopLng_right,stopChanged_right,nextToLast_right,timeDiff_right,receiveTime_right,nextStopID_actual,nextStopName_actual,eta,hour_of_day,minute_of_hour,day_of_week,eta_seconds,distance
str,i64,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,datetime[ns],str,datetime[ns],cat,i64,i64,i64,f64,bool,i64,str,f64,f64,str,f64,f64,bool,bool,duration[ns],str,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,str,datetime[ns],cat,i64,i64,i64,f64,bool,i64,str,f64,f64,str,f64,f64,bool,bool,duration[ns],datetime[ns],i64,str,duration[ns],i8,i8,i8,i64,f32
"""2L""",3,"""Old_1501""",329,41.93639,-88.7736,0,0,33,0,465,1145,10482,271,433,10481,"""19:00:00-2""",true,-2.0,11414,2024-09-01 00:04:41,"""354676050595540""",2024-09-01 00:04:48,"""Outbound""",4,433,465,0.0,true,538,"""Stadium Dr. at New Hall""",41.937874,-88.775536,"""Lincoln Hall""",41.936333,-88.774391,true,true,40s,"""2L""",3,329,41.93746,-88.7754,0,0,33,0,466,1146,10483,0,465,10482,"""19:00:00-2""",true,-2.0,11414,"""354676050595540""",2024-09-01 00:05:33,"""Outbound""",5,465,466,0.0,true,619,"""Grant North""",41.938599,-88.777321,"""Stadium Dr. at New Hall""",41.937874,-88.775536,true,true,1m,2024-09-01 00:05:21,465,"""Stadium Dr. at New Hall""",40s,0,4,7,40,0.054808
"""2L""",3,"""1103""",330,41.93639,-88.77435,0,0,0,0,465,1175,10482,270,433,10481,"""19:30:00-2""",true,-2.0,11415,2024-09-01 00:34:35,"""35467605044c796""",2024-09-01 00:34:47,"""Outbound""",4,433,465,0.0,true,3745,"""Stadium Dr. at New Hall""",41.937874,-88.775536,"""Lincoln Hall""",41.936333,-88.774391,true,true,41s,"""2L""",3,330,41.93746,-88.7754,0,0,0,0,466,1176,10483,0,465,10482,"""19:30:00-2""",true,-2.0,11415,"""35467605044c796""",2024-09-01 00:35:18,"""Outbound""",5,465,466,0.0,true,3799,"""Grant North""",41.938599,-88.777321,"""Stadium Dr. at New Hall""",41.937874,-88.775536,true,true,50s,2024-09-01 00:35:16,465,"""Stadium Dr. at New Hall""",41s,0,34,7,41,0.004875
"""2L""",3,"""1103""",331,41.93638,-88.77397,0,0,8,0,465,1204,10482,270,433,10481,"""20:00:00-2""",true,-1.0,11416,2024-09-01 01:03:34,"""35467605044c796""",2024-09-01 01:03:47,"""Outbound""",4,433,465,0.0,true,6850,"""Stadium Dr. at New Hall""",41.937874,-88.775536,"""Lincoln Hall""",41.936333,-88.774391,true,true,51s,"""2L""",3,331,41.93746,-88.7754,0,0,8,0,466,1205,10483,0,465,10482,"""20:00:00-2""",true,-1.0,11416,"""35467605044c796""",2024-09-01 01:04:32,"""Outbound""",5,465,466,0.0,true,6931,"""Grant North""",41.938599,-88.777321,"""Stadium Dr. at New Hall""",41.937874,-88.775536,true,true,1m,2024-09-01 01:04:25,465,"""Stadium Dr. at New Hall""",51s,1,3,7,51,0.029283
"""2L""",3,"""1103""",332,41.93639,-88.77435,0,0,18,0,465,1234,10482,270,433,10481,"""20:30:00-2""",true,-1.0,11417,2024-09-01 01:33:46,"""35467605044c796""",2024-09-01 01:34:02,"""Outbound""",4,433,465,0.0,true,10090,"""Stadium Dr. at New Hall""",41.937874,-88.775536,"""Lincoln Hall""",41.936333,-88.774391,true,true,51s,"""2L""",3,332,41.93778,-88.77539,0,0,18,0,466,1235,10483,0,465,

In [16]:
mega_df['routeID'].unique()

routeID
str
"""12"""
"""2R"""
"""11"""
"""18"""
"""17"""
"""19"""
"""21"""
"""10"""
"""21 Tripper"""


In [17]:
mega_df['routeID', 'equipmentID', 'lastStopID', 'lastStopID_right', 'lastStopName', 'lastStopName_right',  'receiveTime', 'receiveTime_right', 'eta']

routeID,equipmentID,lastStopID,lastStopID_right,lastStopName,lastStopName_right,receiveTime,receiveTime_right,eta
str,str,i64,i64,str,str,datetime[ns],datetime[ns],duration[ns]
"""2L""","""Old_1501""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 00:04:41,2024-09-01 00:05:21,40s
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 00:34:35,2024-09-01 00:35:16,41s
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 01:03:34,2024-09-01 01:04:25,51s
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 01:33:46,2024-09-01 01:34:37,51s
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 02:04:05,2024-09-01 02:04:35,30s
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 02:33:37,2024-09-01 02:34:37,1m
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 03:02:47,2024-09-01 03:03:48,1m 1s
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 03:34:16,2024-09-01 03:35:16,1m
"""2L""","""1103""",433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-09-01 04:04:30,2024-09-01 04:05:40,1m 10s


In [18]:
m_pred_stopIds = mega_df['lastStopID', 'lastStopID_right'].unique().filter(pl.col('lastStopID_right').is_not_null())

lastStopID,lastStopID_right
i64,i64
678,680
724,725
820,465
515,517
497,500
794,793
433,436
473,476
530,532


In [19]:
#Join the subset on the triple key to get the correct prediction
# m_predset = subset.join(
#     preds,
#     on=["equipmentID", "captureTime"],
#     how="inner"
# )

# m_predset = m_predset['lastStopID', 'stopID'].unique()

# m_pred_stopIds = m_pred_stopIds.rename({"lastStopID_right": "stopID"})

# anti_join_set = m_pred_stopIds.join(
#     m_predset,
#     on=["lastStopID", "stopID"],
#     how="anti"
# )

# anti_join_set = anti_join_set.join(stops.rename({"id": "stopID"}).select(["stopID", "name"]).unique(), on="stopID", how="left").rename({"name": "nextStopName"})
# anti_join_set = anti_join_set.join(stops.rename({"id": "lastStopID"}).select(["lastStopID", "name"]).unique(), on="lastStopID", how="left").rename({"name": "lastStopName"})

# anti_join_set = anti_join_set.sort(by='lastStopID')

In [20]:
subset['timeDiff'].describe()

statistic,value
str,str
"""count""","""35421"""
"""null_count""","""4"""
"""mean""","""0:03:59.878377"""
"""min""","""0:00:09"""
"""25%""","""0:00:40"""
"""50%""","""0:01:00"""
"""75%""","""0:01:21"""
"""max""","""4 days, 14:05:28"""


In [21]:
mega_predset = mega_df.join(
    preds,
    left_on=["equipmentID", "captureTime", 'nextStopID_actual'],
    right_on=["equipmentID", "captureTime", 'stopID'],
    how="inner",
    suffix="_pred"
)

mega_predset = mega_predset.with_columns(
    pl.col('receiveTime').dt.convert_time_zone('America/Chicago'),
)

mega_predset = mega_predset.with_columns(
    (pl.col("time") - pl.col("receiveTime")).alias("predictedDiff")
)

mega_predset = mega_predset.with_columns(
    (pl.duration(minutes=pl.col('minutes'))).alias('minutes')
)

mega_predset = mega_predset.with_columns(
    (pl.col("schedule") - pl.col("receiveTime")).alias("scheduleDiff")
)


mega_predset['routeID', 'equipmentID', 'lat', 'lng', 'captureTime', 'stopID', 'lastStopID', 'nextStopID', 'lastStopName', 'nextStopName_actual', 'time', 'status', 'schedule', 'receiveTime', 'predictedDiff', 'timeDiff', 'scheduleDiff', 'minutesDiff', 'minutes', 'eta', 'statusName']

ColumnNotFoundError: "lat" not found

In [22]:
#Add a uniform column for nextStopPatternID
subset = subset.with_columns(
    pl.col("nextPatternStopID").alias("patternStopID")
)

#Join the subset on the triple key to get the correct prediction
predSet = subset.join(
    preds,
    on=["equipmentID", "patternStopID", "captureTime"],
    how="inner"
)

predSet = predSet.with_columns(
    pl.col('receiveTime').dt.convert_time_zone('America/Chicago'),
)

#Subtract receiveTime and scheduleTime to get the time difference
predSet = predSet.with_columns(
    (pl.col("time") - pl.col("receiveTime")).alias("predictedDiff")
)

predSet = predSet.with_columns(
    (pl.col("schedule") - pl.col("receiveTime")).alias("scheduleDiff")
)

predSet = predSet.with_columns(
    (pl.col('predictedDiff') - pl.col('timeDiff')).alias('predictedActualDiff')
)

predSet = predSet.with_columns(
    (pl.duration(minutes=pl.col('minutes'))).alias('minutes')
)

predSet = predSet.with_columns(
    (pl.col('receiveTime') + (pl.col('minutes'))).alias('receiveTimePlusMinutes')
)

predSet = predSet.with_columns(
    (pl.col('time') - pl.col('receiveTimePlusMinutes')).alias('receivePlusMinuteDiff')
) 

predSet = predSet.with_columns(
    (pl.col('timeDiff') - pl.col('minutes')).alias('minutesDiff')
)

#Filter out negative schedule differences
# predSet = predSet.filter(
#     (pl.col("scheduleDiff").dt.total_minutes() >= 0) 
# )

predSet['routeID', 'equipmentID', 'lat', 'lng', 'scheduleNumber', 'captureTime', 'stopID', 'lastStopID', 'nextStopID', 'lastStopName', 'nextStopName', 'time', 'status', 'schedule', 'receiveTime', 'receiveTimePlusMinutes', 'receivePlusMinuteDiff', 'predictedDiff', 'timeDiff', 'scheduleDiff', 'predictedActualDiff', 'minutesDiff', 'minutes', 'statusName']


routeID,equipmentID,lat,lng,scheduleNumber,captureTime,stopID,lastStopID,nextStopID,lastStopName,nextStopName,time,status,schedule,receiveTime,receiveTimePlusMinutes,receivePlusMinuteDiff,predictedDiff,timeDiff,scheduleDiff,predictedActualDiff,minutesDiff,minutes,statusName
str,str,f64,f64,str,datetime[ns],i64,i64,i64,str,str,"datetime[μs, America/Chicago]",str,"datetime[μs, America/Chicago]","datetime[ns, America/Chicago]","datetime[μs, America/Chicago]",duration[μs],duration[μs],duration[ns],duration[μs],duration[μs],duration[μs],duration[μs],str
"""2L""","""Old_1501""",41.93639,-88.7736,"""19:00:00-2""",2024-09-01 00:04:48,465,433,465,"""Lincoln Hall""","""Stadium Dr. at New Hall""",2024-08-31 19:05:00 CDT,"""07:05PM""",2024-08-31 19:03:00 CDT,2024-08-31 19:04:41 CDT,2024-08-31 19:04:41 CDT,19s,19s,40s,-1m -41s,-21s,40s,0µs,"""On Time"""
"""2L""","""Old_1501""",41.93746,-88.7754,"""19:00:00-2""",2024-09-01 00:05:33,466,465,466,"""Stadium Dr. at New Hall""","""Grant North""",2024-08-31 19:06:00 CDT,"""07:06PM""",2024-08-31 19:04:00 CDT,2024-08-31 19:05:21 CDT,2024-08-31 19:06:21 CDT,-21s,39s,1m,-1m -21s,-21s,0µs,1m,"""On Time"""
"""2L""","""Old_1501""",41.93859,-88.77699,"""19:00:00-2""",2024-09-01 00:06:34,467,466,467,"""Grant North""","""Stevenson North""",2024-08-31 19:06:00 CDT,"""07:06PM""",2024-08-31 19:05:00 CDT,2024-08-31 19:06:21 CDT,2024-08-31 19:07:21 CDT,-1m -21s,-21s,1m 20s,-1m -21s,-1m -41s,20s,1m,"""On Time"""
"""2L""","""Old_1501""",41.93785,-88.77995,"""19:00:00-2""",2024-09-01 00:07:50,468,467,468,"""Stevenson North""","""Stevenson South""",2024-08-31 19:08:00 CDT,"""07:08PM""",2024-08-31 19:06:00 CDT,2024-08-31 19:07:41 CDT,2024-08-31 19:07:41 CDT,19s,19s,51s,-1m -41s,-32s,51s,0µs,"""On Time"""
"""2L""","""Old_1501""",41.93645,-88.78108,"""19:00:00-2""",2024-09-01 00:08:35,436,468,436,"""Stevenson South""","""Convocation Center Entrance 3""",2024-08-31 19:09:00 CDT,"""07:09PM""",2024-08-31 19:07:00 CDT,2024-08-31 19:08:32 CDT,2024-08-31 19:08:32 CDT,28s,28s,40s,-1m -32s,-12s,40s,0µs,"""On Time"""
"""2L""","""Old_1501""",41.93643,-88.78561,"""19:00:00-2""",2024-09-01 00:09:20,437,436,437,"""Convocation Center Entrance 3""","""Lot C3 at Presidents Blvd""",2024-08-31 19:10:00 CDT,"""07:10PM""",2024-08-31 19:08:00 CDT,2024-08-31 19:09:12 CDT,2024-08-31 19:10:12 CDT,-12s,48s,30s,-1m -12s,18s,-30s,1m,"""On Time"""
"""2L""","""Old_1501""",41.93696,-88.78789,"""19:00:00-2""",2024-09-01 00:09:51,438,437,438,"""Lot C3 at Presidents Blvd""","""Norther View Circle""",2024-08-31 19:10:00 CDT,"""07:10PM""",2024-08-31 19:10:00 CDT,2024-08-31 19:09:42 CDT,2024-08-31 19:10:42 CDT,-42s,18s,30s,18s,-12s,-30s,1m,"""On Time"""
"""2L""","""Old_1501""",41.93976,-88.78796,"""19:00:00-2""",2024-09-01 00:10:21,439,438,439,"""Norther View Circle""","""Lincoln Hwy. at High Meadows""",2024-08-31 19:11:00 CDT,"""On Time""",2024-08-31 19:11:00 CDT,2024-08-31 19:10:12 CDT,2024-08-31 19:11:12 CDT,-12s,48s,2m 20s,48s,-1m -32s,1m 20s,1m,"""On Time"""
"""2L""","""Old_1501""",41.93068,-88.77951,"""19:00:00-2""",2024-09-01 00:12:37,440,439,440,"""Lincoln Hwy. at High Meadows""","""Lincoln Hwy. at Culvers""",2024-08-31 19:12:00 CDT,"""07:12PM""",2024-08-31 19:12:00 CDT,2024-08-31 19:12:32 CDT,2024-08-31 19:12:32 CDT,-32s,-32s,1m 10s,-32s,-1m -42s,1m 10s,0µs,"""On Time"""


In [23]:
r2_predset = predSet.filter(
    (pl.col("routeID") == "2L") &
    (pl.col("timeDiff") < pl.duration(minutes=20)) &
    (pl.col("predictedDiff") >= pl.duration(minutes=0))
)

routeID,patternID,equipmentID,tripID,lat,lng,load,capacity,eLoad,blockID,nextStopID,nextStopETA,nextPatternStopID,h,lastStopID,lastPatternStopID,scheduleNumber,inService,onSchedule,trainID,receiveTime,aID,captureTime,direction,seq,lastStopExtID,nextStopExtID,nextStopPctProg,atStop,__index_level_0__,nextStopName,lastStopName,stopChanged,nextToLast,timeDiff,patternStopID,stopID,timePoint,minutes,time,status,schedule,scheduleNumber_right,statuscolor,direction_right,routeID_right,receiveTime_right,__index_level_0___right,statusName,scheduleStr,timeStr,predictedDiff,scheduleDiff,predictedActualDiff,receiveTimePlusMinutes,receivePlusMinuteDiff,minutesDiff
str,i64,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,bool,f64,i64,"datetime[ns, America/Chicago]",str,datetime[ns],cat,i64,i64,i64,f64,bool,i64,str,str,bool,bool,duration[ns],i64,i64,bool,duration[μs],"datetime[μs, America/Chicago]",str,"datetime[μs, America/Chicago]",str,cat,cat,i64,"datetime[ns, America/Chicago]",i64,str,str,str,duration[μs],duration[μs],duration[μs],"datetime[μs, America/Chicago]",duration[μs],duration[μs]
"""2L""",3,"""Old_1501""",329,41.93639,-88.7736,0,0,33,0,465,1145,10482,271,433,10481,"""19:00:00-2""",true,-2.0,11414,2024-08-31 19:04:41 CDT,"""354676050595540""",2024-09-01 00:04:48,"""Outbound""",4,433,465,0.0,true,538,"""Stadium Dr. at New Hall""","""Lincoln Hall""",true,true,40s,10482,465,false,0µs,2024-08-31 19:05:00 CDT,"""07:05PM""",2024-08-31 19:03:00 CDT,"""19:00:00-2""","""#39B139""","""Outbound""",3,2024-08-31 19:04:41 CDT,740,"""On Time""","""07:03PM""","""07:05PM""",19s,-1m -41s,-21s,2024-08-31 19:04:41 CDT,19s,40s
"""2L""",3,"""Old_1501""",329,41.93746,-88.7754,0,0,33,0,466,1146,10483,0,465,10482,"""19:00:00-2""",true,-2.0,11414,2024-08-31 19:05:21 CDT,"""354676050595540""",2024-09-01 00:05:33,"""Outbound""",5,465,466,0.0,true,619,"""Grant North""","""Stadium Dr. at New Hall""",true,true,1m,10483,466,false,1m,2024-08-31 19:06:00 CDT,"""07:06PM""",2024-08-31 19:04:00 CDT,"""19:00:00-2""","""#39B139""","""Outbound""",3,2024-08-31 19:05:21 CDT,845,"""On Time""","""07:04PM""","""07:06PM""",39s,-1m -21s,-21s,2024-08-31 19:06:21 CDT,-21s,0µs
"""2L""",3,"""Old_1501""",329,41.93785,-88.77995,0,0,33,0,468,1148,10485,270,467,10484,"""19:00:00-2""",true,-2.0,11414,2024-08-31 19:07:41 CDT,"""354676050595540""",2024-09-01 00:07:50,"""Outbound""",7,467,468,3.04295,true,862,"""Stevenson South""","""Stevenson North""",true,true,51s,10485,468,false,0µs,2024-08-31 19:08:00 CDT,"""07:08PM""",2024-08-31 19:06:00 CDT,"""19:00:00-2""","""#39B139""","""Outbound""",3,2024-08-31 19:07:41 CDT,1160,"""On Time""","""07:06PM""","""07:08PM""",19s,-1m -41s,-32s,2024-08-31 19:07:41 CDT,19s,51s
"""2L""",3,"""Old_1501""",329,41.93645,-88.78108,0,0,33,0,436,1149,10486,182,468,10485,"""19:00:00-2""",true,-2.0,11414,2024-08-31 19:08:32 CDT,"""354676050595540""",2024-09-01 00:08:35,"""Outbound""",8,468,436,0.0,true,943,"""Convocation Center Entrance 3""","""Stevenson South""",true,true,40s,10486,436,false,0µs,2024-08-31 19:09:00 CDT,"""07:09PM""",2024-08-31 19:07:00 CDT,"""19:00:00-2""","""#39B139""","""Outbound""",3,2024-08-31 19:08:32 CDT,1265,"""On Time""","""07:07PM""","""07:09PM""",28s,-1m -32s,-12s,2024-08-31 19:08:32 CDT,28s,40s
"""2L""",3,"""Old_1501""",329,41.93643,-88.78561,0,0,33,0,437,1150,10487,268,436,10486,"""19:00:00-2""",true,-2.0,11414,2024-08-31 19:09:12 CDT,"""354676050595540""",2024-09-01 00:09:20,"""Outbound""",9,436,437,14.69113,true,1024,"""Lot C3 at Presidents Blvd""","""Convocation Center Entrance 3""",true,true,30s,10487,437,false,1m,2024-08-31 19:10:00 CDT,"""07:10PM""",2024-08-31 19:08:00 CDT,"""19:00:00-2""","""#39B139""","""Outbound""",3,2024-08-31 19:09:12 CDT,1370,"""On Time""","""07:08PM""","""07:10PM""",48s,-1m -12s,18s,2024-08-31 19:10:12 CDT,-12s,-30s
"""2L""",3,"""Old_1501""",329,41.93696,-88.78789,0,0,33,0,438,1151,10488,360,437,10487,"""19:00:00-2""",true,-1.0,11414,2024-08-31 19:09:42 CDT,"""354676050595540""",2024-09-01 00:09

In [25]:
r2_predset['predictedDiff'].describe()

statistic,value
str,str
"""count""","""26043"""
"""null_count""","""0"""
"""mean""","""0:00:40.072802"""
"""min""","""0:00:00"""
"""25%""","""0:00:16"""
"""50%""","""0:00:33"""
"""75%""","""0:00:53"""
"""max""","""0:04:46"""


In [26]:
r2_predset['timeDiff'].describe()

statistic,value
str,str
"""count""","""26043"""
"""null_count""","""0"""
"""mean""","""0:01:22.154820"""
"""min""","""0:00:09"""
"""25%""","""0:00:40"""
"""50%""","""0:01:00"""
"""75%""","""0:01:20"""
"""max""","""0:19:45"""


In [33]:
#Calcuate Metrics
rmse = np.sqrt(mean_squared_error(r2_predset['timeDiff'], r2_predset['predictedDiff']))
mae = mean_absolute_error(r2_predset['timeDiff'], r2_predset['predictedDiff'])
r2 = r2_score(r2_predset['timeDiff'], r2_predset['predictedDiff'])
print(f"R2 Score: {r2}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R2 Score: -0.7193898428080201
RMSE: 126950586240.63504
MAE: 82114748070.4988


In [ ]:
r2_predset['predictedDiff'].describe()

In [ ]:
predSet['scheduleDiff'].describe()

In [ ]:
predSet['predictedActualDiff'].describe()

In [ ]:
diffSet = predSet.filter(pl.col('scheduleDiff') < pl.duration(minutes=-30))

diffSet['equipmentID', 'routeID', 'lastStopName', 'nextStopName', 'receiveTime', 'captureTime', 'schedule', 'time', 'scheduleDiff', 'timeDiff', 'predictedDiff', 'minutesDiff', 'predictedActualDiff', 'minutes']

In [ ]:
outset = predSet.filter(pl.col("predictedDiff") < pl.duration(seconds=-30))

outset['equipmentID', 'routeID', 'receiveTime', 'captureTime', 'schedule', 'time', 'timeDiff', 'predictedDiff', 'minutesDiff', 'predictedActualDiff', 'minutes']

In [ ]:
d = pl.lit(str(datetime.date.today()))

data = preds[:2000]

data

In [ ]:
#Check if the time and status time match or are on time
#The Dateframe should be empty
preds.filter(
    (pl.col("time") != pl.col("status")) &
    (pl.col("status") != "On Time")
)

In [ ]:
#Check if all the status color change togther
(preds.group_by("equipmentID", "captureTime")).agg(pl.col("statuscolor").n_unique())['statuscolor'].value_counts()

In [ ]:
predSet['scheduleDiff', 'statusName'].describe()

In [ ]:
#Check if our statusName column is correct by comparing it to the status column
data.group_by("statusName").agg(
    pl.col("scheduleDiff").min().alias("scheduleDiffMin"),
    pl.col("scheduleDiff").max().alias("scheduleDiffMax"),
    pl.col("scheduleDiff").mean().alias("scheduleDiffMean"),
    pl.col("scheduleDiff").median().alias("scheduleDiffMedian"),
)

In [ ]:
#Pull data for on time and little late statuses
on_time_data = data.filter(pl.col('statusName') == 'On Time')
little_late_data = data.filter(pl.col('statusName') == 'Little Late')

#Convert the scheduleDiff to scheduleDiff_seconds
on_time_data = on_time_data.with_columns(
    (pl.col("scheduleDiff").dt.total_seconds()).alias("scheduleDiff_seconds") 
)

little_late_data = little_late_data.with_columns(
    (pl.col("scheduleDiff").dt.total_seconds()).alias("scheduleDiff_seconds") 
)


# Plot the histograms
plt.figure(figsize=(15, 6))
plt.hist(on_time_data['scheduleDiff_seconds'].to_list(), bins=50, alpha=0.5, width=5, label='On time Schedule Diff')
plt.hist(little_late_data['scheduleDiff_seconds'].to_list(), bins=50, alpha=0.5, width=5, label='Little Late Schedule Diff')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of On-time Schedule Diff')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Define a threshold for overlapping data in minutes
threshold = 5.0 * 60

# Filter out pairs where the difference in scheduleDiff_minutes is within the threshold
overlapping_data = data.filter(pl.col('scheduleDiff').dt.total_seconds() >= threshold).sort('scheduleDiff')

overlapping_data['routeID', 'equipmentID', 'receiveTime', 'schedule','statusName', 'scheduleDiff']